# Policy Impacts on Management Timelags in Idaho
#### In this notebook I will work through comparing forest management implementation time lags (sequencing and size) before and after specific natural resource management policies are enacted.
#### Author: Katie Murenbeeld
#### Init Date: 29 Aug 2019
#### Updated: 


In [1]:
# Import the appropriate libraries

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
import seaborn as sns
import simpledbf 
from simpledbf import Dbf5 
from scipy.stats import norm

In [2]:
# Set the directories
raw_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/DATA/FOREST-TREATMENTS/RAW/'
mod_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/DATA/FOREST-TREATMENTS/MOD/'
output_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/OUTPUT/'

In [3]:
# Convert the most up to date timber harvest data from dbf to csv
#dbf = Dbf5(raw_data_dir + 'S_USA_Activity_TimberHarvest_july2019.dbf', codec='utf-8') # Import the dbf and assign the code 
#dbf.to_csv(mod_data_dir + 'USA_TH_JULY2019.csv') # Covert to csv

In [4]:
# Create dataframe from new csv
df = pd.read_csv(mod_data_dir + 'USA_TH_JULY2019.csv').reset_index()
df.head(1)

,index,ADMIN_FORE,ADMIN_REGI,ADMIN_FO_1,PROCLAIMED,ADMIN_DIST,ADMIN_DI_1,HOME_ORG,ACTIVITY_U,SUID,...,DATA_SOU_1,ACCURACY,CRC_VALUE,UK,EDW_INSERT,ETL_MODIFI,REV_DATE,GIS_ACRES,SHAPE_AREA,SHAPE_LEN
0,0,6,5,Lassen National Forest,506,Almanor Ranger District,51,50651,50651,050651T376000076000,...,NaN,0.0,NaN,NaN,2019-07-01,2019-07-01,2018-01-09,25.759,0.000011,0.026648


In [6]:
# Create new data frame for just Idaho National Forests
df2 = df.loc[df['STATE_ABBR']=='ID'].copy().reset_index()
df2.head(5)

,level_0,index,ADMIN_FORE,ADMIN_REGI,ADMIN_FO_1,PROCLAIMED,ADMIN_DIST,ADMIN_DI_1,HOME_ORG,ACTIVITY_U,...,DATA_SOU_1,ACCURACY,CRC_VALUE,UK,EDW_INSERT,ETL_MODIFI,REV_DATE,GIS_ACRES,SHAPE_AREA,SHAPE_LEN
0,4,4,17,1,Nez Perce - Clearwater National Forest,117,Red River Ranger District,5,11705,11705,...,NaN,0.0,NaN,NaN,2019-07-01,2019-07-01,NaN,5.516,0.000003,0.007659
1,14,14,4,1,Idaho Panhandle National Forests,113,Sandpoint Ranger District,6,10406,10406,...,NaN,0.0,NaN,NaN,2019-07-01,2019-07-01,2009-09-01,18.370,0.000009,0.022294
2,36,36,4,1,Idaho Panhandle National Forests,118,St. Maries Ranger District,4,10404,10404,...,NaN,0.0,NaN,NaN,2019-07-01,2019-07-01,2008-04-26,6.176,0.000003,0.006913
3,49,49,4,1,Idaho Panhandle National Forests,106,Wallace Ranger District,1,10401,10401,...,NaN,0.0,NaN,NaN,2019-07-01,2019-07-01,2008-04-26,29.800,0.000014,0.030704
4,50,50,4,1,Idaho Panhandle National Forests,106,Wallace Ranger District,1,10401,10401,...,NaN,0.0,NaN,NaN,2019-07-01,2019-07-01,2008-04-26,29.800,0.000014,0.030704


In [8]:
#Reduce columns for Boise National Forest dataframe
dfid = df2[['ADMIN_FO_1','ADMIN_DIST','NEPA_DOC_N','ACTIVITY_2','ACTIVITY_N','NBR_UNITS_','NBR_UNITS1','DATE_PLANN','DATE_AWARD','DATE_COMPL']].copy().reset_index()
dfid.head(5)

,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL
0,0,Nez Perce - Clearwater National Forest,Red River Ranger District,DEFAULT FOR NOT REQUIRED,4131,Shelterwood Establishment Cut (with or without...,6.0,6.0,1987-07-01,1987-07-01,1987-07-01
1,1,Idaho Panhandle National Forests,Sandpoint Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",32.0,32.0,1968-06-01,1968-06-01,1968-06-01
2,2,Idaho Panhandle National Forests,St. Maries Ranger District,DEFAULT FOR NOT REQUIRED,4113,Stand Clearcut (EA/RH/FH),6.0,6.0,1985-07-01,1985-07-01,1985-07-01
3,3,Idaho Panhandle National Forests,Wallace Ranger District,DEFAULT FOR NOT REQUIRED,4113,Stand Clearcut (EA/RH/FH),18.0,18.0,1969-06-01,1969-06-01,1969-06-01
4,4,Idaho Panhandle National Forests,Wallace Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",29.0,29.0,1953-08-01,1953-08-01,1953-08-01


In [10]:
# Group some of the activities together for later use. Use np.select(conditions, choices, default=). 
# Set your conditions to the ACTIVITY_2 codes, use isin() for when multiple codes will be grouped.

conditions = [
    (dfid['ACTIVITY_2']==4177),
    (dfid['ACTIVITY_2'].isin([4113,4111])),
    (dfid['ACTIVITY_2']==4151),
    (dfid['ACTIVITY_2']==4152),
    (dfid['ACTIVITY_2'].isin([4121,4141,4145,4131,4148,4193,4194,4196,4132])),
    (dfid['ACTIVITY_2'].isin([4231,4232,4210,4211])),
    (dfid['ACTIVITY_2']==4143),
    (dfid['ACTIVITY_2']==4220),
    (dfid['ACTIVITY_2']==4242)
]
choices = ['A2_CLEARCUT','EA_CLEARCUT','UA_SINGLE','UA_GROUP','UA_SHELTER','INT_CUT','OVERSTORE_CUT','COMM_THIN','HARV_NOSTOCK']

In [11]:
# Use np.select() to create a new column with the coded/grouped activities.

dfid['new_code'] = np.select(conditions, choices, default='none')
dfid.head(5) # check the data

,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL,new_code
0,0,Nez Perce - Clearwater National Forest,Red River Ranger District,DEFAULT FOR NOT REQUIRED,4131,Shelterwood Establishment Cut (with or without...,6.0,6.0,1987-07-01,1987-07-01,1987-07-01,UA_SHELTER
1,1,Idaho Panhandle National Forests,Sandpoint Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",32.0,32.0,1968-06-01,1968-06-01,1968-06-01,INT_CUT
2,2,Idaho Panhandle National Forests,St. Maries Ranger District,DEFAULT FOR NOT REQUIRED,4113,Stand Clearcut (EA/RH/FH),6.0,6.0,1985-07-01,1985-07-01,1985-07-01,EA_CLEARCUT
3,3,Idaho Panhandle National Forests,Wallace Ranger District,DEFAULT FOR NOT REQUIRED,4113,Stand Clearcut (EA/RH/FH),18.0,18.0,1969-06-01,1969-06-01,1969-06-01,EA_CLEARCUT
4,4,Idaho Panhandle National Forests,Wallace Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",29.0,29.0,1953-08-01,1953-08-01,1953-08-01,INT_CUT


In [12]:
# Convert DATE_* columns to datetime. Add a new column for the serial dates
bnf_datearrP = dfid['DATE_PLANN'].values
bnf_datearr_dtP = pd.to_datetime(bnf_datearrP)
dfid['SerDatesPlan'] = bnf_datearr_dtP

bnf_datearrA = dfid['DATE_AWARD'].values
bnf_datearr_dtA = pd.to_datetime(bnf_datearrA)
dfid['SerDatesAward'] = bnf_datearr_dtA

bnf_datearrC = dfid['DATE_COMPL'].values
bnf_datearr_dtC = pd.to_datetime(bnf_datearrC)
dfid['SerDatesComp'] = bnf_datearr_dtC

dfid.head(1) # Check the data

,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL,new_code,SerDatesPlan,SerDatesAward,SerDatesComp
0,0,Nez Perce - Clearwater National Forest,Red River Ranger District,DEFAULT FOR NOT REQUIRED,4131,Shelterwood Establishment Cut (with or without...,6.0,6.0,1987-07-01,1987-07-01,1987-07-01,UA_SHELTER,1987-07-01,1987-07-01,1987-07-01


The Healthy Forest Restoration Act of 2003 (HFRA) was enacted to relieve "analysis-paralysis" and allow for quicker implementation of certain forest treatments, specifically intermediate treatments such as salvage, sanitation, improvement, and liberation cuts.

I will assess the changes in the time lag between date when intermediate treatments were planned to be completed and when they were actually completed. I will also compare the size of treatments pre- and post passage of the HFRA.

In [16]:
# Here I will sort out the data so that only items planned to be completed after 2003 and before 2019 
# are included.

#greater than the start date and smaller than the end date
mask = (dfid['SerDatesPlan'] > '2004-01-01') & (dfid['SerDatesPlan'] <= '2018-12-31')

# Create the subset for the range of dates when HFRA would be applied
hfra = dfid.loc[mask].copy()
# Check the date range
print(hfra['SerDatesPlan'].min(), hfra['SerDatesPlan'].max()) # Check the date range.

# Note: this method can be used for other policies of interest, such as NEPA, ESA, and NFMA.

2004-01-14 00:00:00 2018-12-31 00:00:00


In [40]:
# Next I will sort the data by INT_CUT (intermediate cuts)

hfra_ic = hfra.loc[hfra['new_code']=='INT_CUT'].copy().reset_index()
hfra_ic.head(5)

,level_0,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL,new_code,SerDatesPlan,SerDatesAward,SerDatesComp,time_lag,size_diff
0,20,20,Nez Perce - Clearwater National Forest,Elk City Ranger District,AMERICAN AND CROOKED RIVER PROJECT,4231,"Salvage Cut (intermediate treatment, not regen...",25.0,25.0,2006-07-01,2006-11-01,2006-11-01,INT_CUT,2006-07-01,2006-11-01,2006-11-01,123 days,0.0
1,58,58,Payette National Forest,Council Ranger District,(PALS)CROOKED RIVER VEGETATION MANGEMENT PROJECT,4210,Improvement Cut,2.0,2.0,2010-04-16,2010-04-16,2012-02-14,INT_CUT,2010-04-16,2010-04-16,2012-02-14,669 days,0.0
2,90,90,Nez Perce - Clearwater National Forest,Slate Creek Ranger District,JOHN DAY MISTLETOE,4231,"Salvage Cut (intermediate treatment, not regen...",13.0,13.0,2005-08-01,2005-09-15,2005-09-15,INT_CUT,2005-08-01,2005-09-15,2005-09-15,45 days,0.0
3,134,134,NaN,Pierce Ranger District,"(PALS)NEZ PERCE-CLEARWATER NF'S ROADSIDE, RECR...",4231,"Salvage Cut (intermediate treatment, not regen...",1.5,1.5,2016-07-01,2016-10-02,2016-10-02,INT_CUT,2016-07-01,2016-10-02,2016-10-02,93 days,0.0
4,334,334,Boise National Forest,Emmett Ranger District,(PALS)SCRIVER CREEK INTEGRATED RESTORATION PRO...,4210,Improvement Cut,21.0,21.0,2013-09-26,2013-09-26,NaN,INT_CUT,2013-09-26,2013-09-26,NaT,NaT,0.0


In [41]:
# Next I will calculate the difference between the planned and completed dates for the
# Intermediate Cuts.

hfra_ic['time_lag']=hfra_ic['SerDatesComp'] - hfra_ic['SerDatesPlan']
hfra_ic.head(5)

,level_0,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL,new_code,SerDatesPlan,SerDatesAward,SerDatesComp,time_lag,size_diff
0,20,20,Nez Perce - Clearwater National Forest,Elk City Ranger District,AMERICAN AND CROOKED RIVER PROJECT,4231,"Salvage Cut (intermediate treatment, not regen...",25.0,25.0,2006-07-01,2006-11-01,2006-11-01,INT_CUT,2006-07-01,2006-11-01,2006-11-01,123 days,0.0
1,58,58,Payette National Forest,Council Ranger District,(PALS)CROOKED RIVER VEGETATION MANGEMENT PROJECT,4210,Improvement Cut,2.0,2.0,2010-04-16,2010-04-16,2012-02-14,INT_CUT,2010-04-16,2010-04-16,2012-02-14,669 days,0.0
2,90,90,Nez Perce - Clearwater National Forest,Slate Creek Ranger District,JOHN DAY MISTLETOE,4231,"Salvage Cut (intermediate treatment, not regen...",13.0,13.0,2005-08-01,2005-09-15,2005-09-15,INT_CUT,2005-08-01,2005-09-15,2005-09-15,45 days,0.0
3,134,134,NaN,Pierce Ranger District,"(PALS)NEZ PERCE-CLEARWATER NF'S ROADSIDE, RECR...",4231,"Salvage Cut (intermediate treatment, not regen...",1.5,1.5,2016-07-01,2016-10-02,2016-10-02,INT_CUT,2016-07-01,2016-10-02,2016-10-02,93 days,0.0
4,334,334,Boise National Forest,Emmett Ranger District,(PALS)SCRIVER CREEK INTEGRATED RESTORATION PRO...,4210,Improvement Cut,21.0,21.0,2013-09-26,2013-09-26,NaN,INT_CUT,2013-09-26,2013-09-26,NaT,NaT,0.0


In [42]:
# Print the summary statistics for the time lag
hfra_ic['time_lag'].describe()

count                         822
mean     396 days 01:17:04.817518
std      565 days 10:05:30.363411
min               0 days 00:00:00
25%               9 days 00:00:00
50%             151 days 00:00:00
75%             541 days 00:00:00
max            2897 days 00:00:00
Name: time_lag, dtype: object

In [43]:
# Next I will calculate the difference between the size of the planned and completed
# Intermediate Cuts.

hfra_ic['size_diff']=hfra_ic['NBR_UNITS1'] - hfra_ic['NBR_UNITS_']
hfra_ic.head(5)

,level_0,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL,new_code,SerDatesPlan,SerDatesAward,SerDatesComp,time_lag,size_diff
0,20,20,Nez Perce - Clearwater National Forest,Elk City Ranger District,AMERICAN AND CROOKED RIVER PROJECT,4231,"Salvage Cut (intermediate treatment, not regen...",25.0,25.0,2006-07-01,2006-11-01,2006-11-01,INT_CUT,2006-07-01,2006-11-01,2006-11-01,123 days,0.0
1,58,58,Payette National Forest,Council Ranger District,(PALS)CROOKED RIVER VEGETATION MANGEMENT PROJECT,4210,Improvement Cut,2.0,2.0,2010-04-16,2010-04-16,2012-02-14,INT_CUT,2010-04-16,2010-04-16,2012-02-14,669 days,0.0
2,90,90,Nez Perce - Clearwater National Forest,Slate Creek Ranger District,JOHN DAY MISTLETOE,4231,"Salvage Cut (intermediate treatment, not regen...",13.0,13.0,2005-08-01,2005-09-15,2005-09-15,INT_CUT,2005-08-01,2005-09-15,2005-09-15,45 days,0.0
3,134,134,NaN,Pierce Ranger District,"(PALS)NEZ PERCE-CLEARWATER NF'S ROADSIDE, RECR...",4231,"Salvage Cut (intermediate treatment, not regen...",1.5,1.5,2016-07-01,2016-10-02,2016-10-02,INT_CUT,2016-07-01,2016-10-02,2016-10-02,93 days,0.0
4,334,334,Boise National Forest,Emmett Ranger District,(PALS)SCRIVER CREEK INTEGRATED RESTORATION PRO...,4210,Improvement Cut,21.0,21.0,2013-09-26,2013-09-26,NaN,INT_CUT,2013-09-26,2013-09-26,NaT,NaT,0.0


In [44]:
# Print the summary statistics for the difference in size
hfra_ic['size_diff'].describe()

count    1100.000000
mean       -1.479091
std         8.047404
min       -96.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        12.000000
Name: size_diff, dtype: float64

In [53]:
# Now I will repeat the process for pre-HFRA implementation

#greater than the start date and smaller than the end date
mask2 = (dfid['SerDatesPlan'] >= '1967-01-01') & (dfid['SerDatesPlan'] < '2004-01-01')

# Create the subset for the range of dates when HFRA would be applied
prehfra = dfid.loc[mask2].copy()
# Check the date range
print(prehfra['SerDatesPlan'].min(), prehfra['SerDatesPlan'].max()) # Check the date range.



1967-01-01 00:00:00 2003-12-01 00:00:00


In [54]:
# Next I will sort the data by INT_CUT (intermediate cuts)

prehfra_ic = prehfra.loc[prehfra['new_code']=='INT_CUT'].copy().reset_index()
prehfra_ic.head(5)

,level_0,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL,new_code,SerDatesPlan,SerDatesAward,SerDatesComp
0,1,1,Idaho Panhandle National Forests,Sandpoint Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",32.0,32.0,1968-06-01,1968-06-01,1968-06-01,INT_CUT,1968-06-01,1968-06-01,1968-06-01
1,18,18,NaN,Powell Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",29.0,29.0,1976-06-01,1976-06-01,1976-06-01,INT_CUT,1976-06-01,1976-06-01,1976-06-01
2,25,25,Idaho Panhandle National Forests,Fernan Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",1.0,1.0,2002-08-01,2002-08-01,2002-08-01,INT_CUT,2002-08-01,2002-08-01,2002-08-01
3,27,27,Nez Perce - Clearwater National Forest,Slate Creek Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",2.0,2.0,1995-07-01,1995-07-01,1995-07-01,INT_CUT,1995-07-01,1995-07-01,1995-07-01
4,28,28,NaN,North Fork Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",11.0,11.0,1985-10-01,1985-10-01,1985-10-01,INT_CUT,1985-10-01,1985-10-01,1985-10-01


In [55]:
# Next I will calculate the difference between the planned and completed dates for the
# Intermediate Cuts.

prehfra_ic['time_lag']=prehfra_ic['SerDatesComp'] - prehfra_ic['SerDatesPlan']
prehfra_ic.head(5)

,level_0,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL,new_code,SerDatesPlan,SerDatesAward,SerDatesComp,time_lag
0,1,1,Idaho Panhandle National Forests,Sandpoint Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",32.0,32.0,1968-06-01,1968-06-01,1968-06-01,INT_CUT,1968-06-01,1968-06-01,1968-06-01,0 days
1,18,18,NaN,Powell Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",29.0,29.0,1976-06-01,1976-06-01,1976-06-01,INT_CUT,1976-06-01,1976-06-01,1976-06-01,0 days
2,25,25,Idaho Panhandle National Forests,Fernan Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",1.0,1.0,2002-08-01,2002-08-01,2002-08-01,INT_CUT,2002-08-01,2002-08-01,2002-08-01,0 days
3,27,27,Nez Perce - Clearwater National Forest,Slate Creek Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",2.0,2.0,1995-07-01,1995-07-01,1995-07-01,INT_CUT,1995-07-01,1995-07-01,1995-07-01,0 days
4,28,28,NaN,North Fork Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",11.0,11.0,1985-10-01,1985-10-01,1985-10-01,INT_CUT,1985-10-01,1985-10-01,1985-10-01,0 days


In [56]:
# Next I will calculate the difference between the size of the planned and completed
# Intermediate Cuts.

prehfra_ic['size_diff']=prehfra_ic['NBR_UNITS1'] - prehfra_ic['NBR_UNITS_']
prehfra_ic.head(5)

,level_0,index,ADMIN_FO_1,ADMIN_DIST,NEPA_DOC_N,ACTIVITY_2,ACTIVITY_N,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_AWARD,DATE_COMPL,new_code,SerDatesPlan,SerDatesAward,SerDatesComp,time_lag,size_diff
0,1,1,Idaho Panhandle National Forests,Sandpoint Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",32.0,32.0,1968-06-01,1968-06-01,1968-06-01,INT_CUT,1968-06-01,1968-06-01,1968-06-01,0 days,0.0
1,18,18,NaN,Powell Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",29.0,29.0,1976-06-01,1976-06-01,1976-06-01,INT_CUT,1976-06-01,1976-06-01,1976-06-01,0 days,0.0
2,25,25,Idaho Panhandle National Forests,Fernan Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",1.0,1.0,2002-08-01,2002-08-01,2002-08-01,INT_CUT,2002-08-01,2002-08-01,2002-08-01,0 days,0.0
3,27,27,Nez Perce - Clearwater National Forest,Slate Creek Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",2.0,2.0,1995-07-01,1995-07-01,1995-07-01,INT_CUT,1995-07-01,1995-07-01,1995-07-01,0 days,0.0
4,28,28,NaN,North Fork Ranger District,DEFAULT FOR NOT REQUIRED,4231,"Salvage Cut (intermediate treatment, not regen...",11.0,11.0,1985-10-01,1985-10-01,1985-10-01,INT_CUT,1985-10-01,1985-10-01,1985-10-01,0 days,0.0


In [59]:
# Print the summary statistics for the time lag and the difference in size
# pre HFRA intermediate cuts.
print(prehfra_ic['time_lag'].describe(),
prehfra_ic['size_diff'].describe())

count                       11984
mean      95 days 20:57:06.969292
std      365 days 17:04:27.421490
min           -223 days +00:00:00
25%               0 days 00:00:00
50%               0 days 00:00:00
75%               0 days 00:00:00
max            9131 days 00:00:00
Name: time_lag, dtype: object count    11988.000000
mean         0.000918
std          0.969674
min        -30.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         90.000000
Name: size_diff, dtype: float64


In [64]:
# Create a table/data frame of the summary stats (ss) for the time lag and size
#Pre HFRA intermediate cuts
prehfra_ss = prehfra_ic[['time_lag', 'size_diff']].describe().reset_index()
prehfra_ss

,index,time_lag,size_diff
0,count,11984,11988.000000
1,mean,95 days 20:57:06.969292,0.000918
2,std,365 days 17:04:27.421490,0.969674
3,min,-223 days +00:00:00,-30.000000
4,25%,0 days 00:00:00,0.000000
5,50%,0 days 00:00:00,0.000000
6,75%,0 days 00:00:00,0.000000
7,max,9131 days 00:00:00,90.000000


In [65]:
# Create a table/data frame of the summary stats (ss) for the time lag and size
#Post HFRA intermediate cuts
hfra_ss = hfra_ic[['time_lag', 'size_diff']].describe().reset_index()
hfra_ss

,index,time_lag,size_diff
0,count,822,1100.000000
1,mean,396 days 01:17:04.817518,-1.479091
2,std,565 days 10:05:30.363411,8.047404
3,min,0 days 00:00:00,-96.000000
4,25%,9 days 00:00:00,0.000000
5,50%,151 days 00:00:00,0.000000
6,75%,541 days 00:00:00,0.000000
7,max,2897 days 00:00:00,12.000000
